In [1]:
#Importing Libraries
import pandas as pd
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import scale
import numpy as np

In [2]:
#Read the twitter dataset from the file
data = pd.read_csv('dataset.csv',index_col=False)
print(data.shape)
data.head(6)

context_data = pd.read_csv('context.csv',index_col=False)
context_data = context_data.replace(np.nan, '', regex=True)
print(context_data.shape)
context_data.head(6)

(99996, 2)
(99996, 2)
(99996, 2)
(99996, 2)


,tweet_id,context
0,1,
1,2,RT @Papapishu: Man it would fucking rule if we...
2,3,
3,4,
4,5,
5,6,RT @Vitiligoprince: Hate Being sexually Frustr...


,tweet_id,context
0,1,
1,2,RT @Papapishu: Man it would fucking rule if we...
2,3,
3,4,
4,5,
5,6,RT @Vitiligoprince: Hate Being sexually Frustr...


In [3]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(data.tweet,data.label,test_size = 0.2)

ContextX_train, ContextX_test = train_test_split(context_data.context,test_size = 0.2)

In [4]:
# Clean the text by removing ...

def to_lower(word):
    result = word.lower()
    return result

def remove_hyperlink(word):
    return  re.sub(r"http\S+", "", word)

def remove_mentions(word):
    return re.sub(r"@\S+", "", word)

def remove_number(word):
    result = re.sub(r'\d+', '', word)
    return result

def remove_punctuation(word):
    result = re.sub('[^A-Za-z]+', ' ', word)
    return result

def remove_whitespace(word):
    result = word.strip()
    return result

def replace_newline(word):
    return word.replace('\n','')

def remove_stopwords(word):
    stopwords = nltk.corpus.stopwords.words('english')
    return ' '.join(word for word in i.split() if word not in stopwords)

def clean_up_pipeline(sentence):
    cleaning_data = [remove_hyperlink,
                      replace_newline,
                      to_lower,
                      remove_number,
                      remove_punctuation,
                      remove_whitespace]
    for func in cleaning_data:
        
        sentence = func(sentence)
    return sentence


X_train = X_train.apply(clean_up_pipeline)
X_test = X_test.apply(clean_up_pipeline)
print(X_train[:3])
print(X_train.shape)
print(X_test[:3])
print(X_test.shape)

ContextX_train = ContextX_train.apply(clean_up_pipeline)
ContextX_test = ContextX_test.apply(clean_up_pipeline)
print(ContextX_train[:3])
print(ContextX_train.shape)
print(ContextX_test[:3])
print(ContextX_test.shape)

78828    we are all affected by the destruction of our ...
44043    shaktisinhgohil adani group s asset are perfor...
83830    she s finishing a vodka lemonade i think he s ...
Name: tweet, dtype: object
(79996,)
65291    as if jermainejunior hadn t played the victim ...
71500    my child wont be have to ask when daddy coming...
9053     you look like a keeper can i have your number ...
Name: tweet, dtype: object
(20000,)
78828    we are all affected by the destruction of our ...
44043    shaktisinhgohil adani group s asset are perfor...
83830    she s finishing a vodka lemonade i think he s ...
Name: tweet, dtype: object
(79996,)
65291    as if jermainejunior hadn t played the victim ...
71500    my child wont be have to ask when daddy coming...
9053     you look like a keeper can i have your number ...
Name: tweet, dtype: object
(20000,)
3193     
22599    
53266    
Name: context, dtype: object
(79996,)
41996    
60170    
34294    
Name: context, dtype: object
(20000,)
3193     
22

# Tokenization

In [5]:
# X_train = X_train.apply(word_tokenize)
# X_test = X_test.apply(word_tokenize)
print(X_train[:3])
print(X_train.shape)
print(X_test[:3])
print(X_test.shape)

78828    we are all affected by the destruction of our ...
44043    shaktisinhgohil adani group s asset are perfor...
83830    she s finishing a vodka lemonade i think he s ...
Name: tweet, dtype: object
(79996,)
65291    as if jermainejunior hadn t played the victim ...
71500    my child wont be have to ask when daddy coming...
9053     you look like a keeper can i have your number ...
Name: tweet, dtype: object
(20000,)
78828    we are all affected by the destruction of our ...
44043    shaktisinhgohil adani group s asset are perfor...
83830    she s finishing a vodka lemonade i think he s ...
Name: tweet, dtype: object
(79996,)
65291    as if jermainejunior hadn t played the victim ...
71500    my child wont be have to ask when daddy coming...
9053     you look like a keeper can i have your number ...
Name: tweet, dtype: object
(20000,)


# Stemming

In [6]:
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text])

X_train = X_train.apply(lambda text: stem_words(text))
X_test = X_test.apply(lambda text: stem_words(text))

ContextX_train = ContextX_train.apply(lambda text: stem_words(text))
ContextX_test = ContextX_test.apply(lambda text: stem_words(text))

In [7]:
# from gensim.models.word2vec import Word2Vec 
# #Word Embedding using Word2Vec
# wordvector = Word2Vec(vector_size=300, min_count=10)
# wordvector.build_vocab([x for x in X_train])
# wordvector.train([x for x in X_train],total_examples=wordvector.corpus_count, epochs=1)

In [8]:
# wordvector.wv['a']

In [9]:
# from gensim.models import KeyedVectors
# # Load vectors directly from the file
# model = KeyedVectors.load_word2vec_format('data/GoogleGoogleNews-vectors-negative300.bin', binary=True)
# # Access vectors for specific words with a keyed lookup:
# vector = model['easy']
# # see the shape of the vector (300,)
# vector.shape
# # Processing sentences is not as simple as with Spacy:
# vectors = [model[x] for x in "This is some text I am processing with Spacy".split(' ')]

In [10]:
import gensim.downloader as api
# path = api.load("word2vec-google-news-300", return_path=True)
# print(path)

/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
import gensim
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
# glove_vectors['a']

In [ ]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += glove_vectors[word].reshape((1, size)) #* tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
from tqdm import tqdm
import numpy as np
tqdm.pandas(desc="progress-bar")

train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in tqdm(map(lambda x: x, X_train))])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

context_train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in tqdm(map(lambda x: x, ContextX_train))])
context_train_vecs_w2v = scale(train_vecs_w2v)

context_test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, ContextX_test)])
context_test_vecs_w2v = scale(test_vecs_w2v)

In [ ]:
print(y_train)
from keras.utils import to_categorical
encoded = to_categorical(y_train)
encoded

In [ ]:
from keras.layers import Input, Dense, Embedding, Convolution1D, MaxPooling1D
from keras.layers import Flatten, Concatenate
from keras.models import Model
from keras.optimizers import Adam

class WordCNN_Ctxt(object):
    def __init__(
            self, sequence_length, n_classes, vocab_size,
            filter_sizes, num_filters, learning_rate=0.001, 
            embedding_size=300, embedding_matrix=None, train_embedding=True):

        inputs = Input(shape=(sequence_length,))
        ctxt_inputs = Input(shape=(sequence_length,))

        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_size,
                                    trainable=train_embedding, weights=[embedding_matrix])(inputs)
        ctxt_embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_size,
                                    trainable=train_embedding, weights=[embedding_matrix])(ctxt_inputs)

        conv_blocks = []
        for filter_size in filter_sizes:
            conv = Convolution1D(filters=num_filters, kernel_size=filter_size,
                                 padding="valid", activation="relu", strides=1)(embedding_layer)
            conv = MaxPooling1D(pool_size=sequence_length - filter_size + 1)(conv)
            conv = Flatten()(conv)
            conv_blocks.append(conv)

        ctxt_conv_blocks = []
        for filter_size in filter_sizes:
            ctxt_conv = Convolution1D(filters=num_filters, kernel_size=filter_size,
                                 padding="valid", activation="relu", strides=1)(ctxt_embedding_layer)
            ctxt_conv = MaxPooling1D(pool_size=sequence_length - filter_size + 1)(ctxt_conv)
            ctxt_conv = Flatten()(ctxt_conv)
            ctxt_conv_blocks.append(ctxt_conv)

        cnn = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
        ctxt_cnn = Concatenate()(ctxt_conv_blocks) if len(ctxt_conv_blocks) > 1 else ctxt_conv_blocks[0]

        cnn = Dense(9, activation="relu")(cnn)
        ctxt_cnn = Dense(6, activation="relu")(ctxt_cnn) # Reduce dimension for context inputs.

        merged_net = Concatenate()([ctxt_cnn, cnn])

        output = Dense(n_classes, activation="softmax")(merged_net)

        self.model = Model([inputs,ctxt_inputs], output)

        self.model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learning_rate), metrics=["accuracy"])
        self.model.summary()
        
sequence_length = train_vecs_w2v.shape[1]
n_classes = 4
vocab_size = 3000000
filter_sizes = [3]
num_filters = 32
        
model = WordCNN_Ctxt(sequence_length=sequence_length, n_classes=n_classes, vocab_size=vocab_size, 
filter_sizes=filter_sizes, num_filters=num_filters, embedding_size=300)

model.model.fit(x=[train_vecs_w2v, context_train_vecs_w2v], y=encoded, batch_size=32, verbose=2, epochs=300)

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import *
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=300))
model.add(Dense(5, activation='softmax'))
# adam=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer="adam",
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])

model.fit(train_vecs_w2v, encoded, epochs=300, batch_size=32, verbose=2)

In [ ]:
encoded_y_test = to_categorical(y_test)
encoded_y_test
score = model.evaluate(test_vecs_w2v, encoded_y_test, verbose=2)
print(score[1])

In [ ]:
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# params={}
# params['learning_rate']=0.03
# params['boosting_type']='gbdt' #GradientBoostingDecisionTree
# params['objective']='multiclass' #Multi-class target feature
# params['metric']='multi_logloss' #metric for multi-class
# params['max_depth']=10
# params['num_class']=5 #no.of unique values in the target class not inclusive of the end value

# OLGBM =LGBMClassifier(**params)
SLGBM =LGBMClassifier(n_estimators =1000,  n_jobs=-1,verbose=1)
clf = make_pipeline(sc,SLGBM)
clf.fit(train_vecs_w2v, y_train)

In [ ]:
def Predict_Result(y, predicted,predicted_proba):
    
	confusion = confusion_matrix(y, predicted)

	TP = confusion[1, 1]
	TN = confusion[0, 0]
	FP = confusion[0, 1]
	FN = confusion[1, 0]

	# Specificity
	SPE_cla = (TN/float(TN+FP))*100 

	# False Positive Rate
	FPR = (FP/float(TN+FP))

	#False Negative Rate (Miss Rate)
	FNR = (FN/float(FN+TP))

	#Balanced Accuracy
	ACC_Bal = 0.5*((TP/float(TP+FN))+(TN/float(TN+FP)))*100

	# compute MCC
	MCC_cla = matthews_corrcoef(y, predicted)
	F1_cla = f1_score(y, predicted)
	PREC_cla = precision_score(y, predicted)*100
	REC_cla = recall_score(y, predicted)*100
	Accuracy_cla = accuracy_score(y, predicted)*100

	ROC_auc_score= roc_auc_score(y, predicted_proba)*100
	PR_auc_score= average_precision_score(y, predicted_proba)*100
	sw = (REC_cla+SPE_cla-1)

	print(f"SN (%), {REC_cla:.2f}")
	print(f"SP (%), {SPE_cla:.2f}")
	print(f"Sw (%), {sw:.2f}")
	print(f"BACC (%), {ACC_Bal:.2f}")
	print(f"MCC, {MCC_cla:.3f}")
	print(f"ACC (%), {Accuracy_cla:.2f}") 
	print(f"FPR, {FPR:.3f}")
	print(f"FNR, {FNR:.3f}")
	print(f"PR (%), {PREC_cla:.2f}")
	print(f"F1-score, {F1_cla:.3f}") 
	print(f"ROCAUC (%), {ROC_auc_score:.2f}")
	print(f"PRAUC (%), {PR_auc_score:.2f}")

In [ ]:
from sklearn.metrics import *
predicted=clf.predict(test_vecs_w2v)
predicted_proba=clf.predict_proba(test_vecs_w2v)
# threshold=0.5
# y_pred = (prob_predict0[:,1] >= threshold).astype(int)
# Predict_Result(y_test.astype(float), np.array(y_pred),prob_predict0[:,1])

In [ ]:
print(prob_predict0)
y=y_test
MCC_cla = matthews_corrcoef(y, predicted)
F1_cla = f1_score(y, predicted,average="micro")
PREC_cla = precision_score(y, predicted,average="micro")*100
REC_cla = recall_score(y, predicted,average="micro")*100
Accuracy_cla = accuracy_score(y, predicted)*100
ROC_auc_score= roc_auc_score(y, predicted_proba,multi_class="ovr")*100

print(f"SN (%), {REC_cla:.2f}")
print(f"MCC, {MCC_cla:.3f}")
print(f"ACC (%), {Accuracy_cla:.2f}") 
print(f"PR (%), {PREC_cla:.2f}")
print(f"F1-score, {F1_cla:.3f}") 
print(f"ROCAUC (%), {ROC_auc_score:.2f}")